<a href="https://colab.research.google.com/github/Data-Science-and-Data-Analytics-Courses/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit/blob/master/Packages/Files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
from contextlib import contextmanager
import requests
from urllib.parse import urlsplit
from pathlib import Path
import io
import tarfile, gzip, zipfile
from IPython.display import display

# General

In [0]:
def download(url, dest=".", name="", force=False, options="-#"):
  """
  Download url into dest
  name: if provided, rename downloaded
  force: if True, overwrite existing resource
  options: supported by curl (https://curl.haxx.se/docs/manpage.html)
  """
  
  rurl = urlsplit(url)
  dest = Path(dest).resolve()
  downloaded = dest / (name or Path(rurl.path).name)
  
  # Download
  if not downloaded.exists() or force:
    !curl $options --create-dirs "{rurl.geturl()}" -o "{downloaded}"
  else:
    print(f"{downloaded} already exists")

  return downloaded

if __name__ == "__main__":
  url="http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz"
  download(url)

/content/train-images-idx3-ubyte.gz already exists


# Tar

In [0]:
@contextmanager
def readtar(fpath, member, mode="r"):
  """
  Read member in fpath
  mode: supported by tarfile.open (https://docs.python.org/3/library/tarfile.html#tarfile.open)
  """

In [0]:
def runtar(url, dest=".", mode="r", peek=False):
  """
  Retrieve url and untar into dest
  mode: supported by tarfile.open (https://docs.python.org/3/library/tarfile.html#tarfile.open)
  peek: if True, show members before extraction
  """

  rurl = urlsplit(url)
  dest = Path(dest).resolve()

  # Retrieve
  r = requests.get(url)
  t = tarfile.open(fileobj=io.BytesIO(r.content), mode=mode)

  # Members
  members = t.getnames()
  if peek:
    display("Extract", members)
    dest = Path(input(f"into {dest} (or specify): ") or dest)
  
  # Extract
  extracted = list(map(dest.joinpath, members))
  t.extractall(dest)

  return extracted

if __name__ == "__main__":
  url = "https://cvml.ist.ac.at/AwA/AwA-base.tar.bz2"
  runtar(url, mode="r:bz2", peek=True)

'Extract'

['Animals_with_Attributes/README-attributes.txt',
 'Animals_with_Attributes/Features/README-features.txt',
 'Animals_with_Attributes/classes.txt',
 'Animals_with_Attributes/predicates.txt',
 'Animals_with_Attributes/testclasses.txt',
 'Animals_with_Attributes/trainclasses.txt',
 'Animals_with_Attributes/predicate-matrix-binary.txt',
 'Animals_with_Attributes/predicate-matrix-continuous.txt',
 'Animals_with_Attributes/predicate-matrix.png',
 'Animals_with_Attributes/lampert-cvpr2009.pdf']

into /content (or specify): 


# Zip

In [0]:
def runzip(url, dest=".", names=None, top=True, force=False, peek=False):
  """
  Retrieve url and unzip names into dest
  top: if True, create top directory if not exist
  force: if True, overwrite existing members
  peek: if True, show members before extraction
  """
  
  rurl = urlsplit(url)
  dest = Path(dest).resolve()

  # Retrieve url
  r = requests.get(url)
  z = zipfile.ZipFile(io.BytesIO(r.content))
    
  # Members to extract
  members = z.namelist()
  if names:
    members = [m for m in members if any(map(Path(m).match, names))]
  
  # Top directory
  tops = (Path(m).parts[0] for m in members) # tops of directory tree
  if top and len(set(tops))>=2:
    dest = dest / Path(rurl.path).stem # filename without .zip

  # Show members
  if peek:
    display("Extract", members)
    dest = Path(input(f"into {dest} (or specify): ") or dest)
  
  # Extract
  extracted = list(map(dest.joinpath, members))
  for m, e in zip(members, extracted):
    if not e.exists() or force:
      z.extract(m, dest)
    else:
      print(f"{e} already exists")
  
  return extracted # paths to extracted/existing

if __name__ == "__main__":
  url = "https://prod-edxapp.edx-cdn.org/assets/courseware/v1/e7b080a506546a859b50d03f9d0705f5/asset-v1:UCSanDiegoX+DSE220x+1T2019+type@asset+block/DSE220x_PA7.zip"
  runzip(url, peek=True)

'Extract'

['DSE220x_PA7/',
 'DSE220x_PA7/kernel_SVM_and_Perceptron.ipynb',
 '__MACOSX/',
 '__MACOSX/DSE220x_PA7/',
 '__MACOSX/DSE220x_PA7/._kernel_SVM_and_Perceptron.ipynb',
 'DSE220x_PA7/_nbgrader.log',
 '__MACOSX/DSE220x_PA7/.__nbgrader.log',
 'DSE220x_PA7/.ipynb_checkpoints/',
 'DSE220x_PA7/.ipynb_checkpoints/kernel_SVM_and_Perceptron-checkpoint.ipynb',
 '__MACOSX/DSE220x_PA7/.ipynb_checkpoints/',
 '__MACOSX/DSE220x_PA7/.ipynb_checkpoints/._kernel_SVM_and_Perceptron-checkpoint.ipynb',
 '__MACOSX/DSE220x_PA7/._.ipynb_checkpoints',
 'DSE220x_PA7/data1.txt',
 '__MACOSX/DSE220x_PA7/._data1.txt',
 'DSE220x_PA7/data3.txt',
 '__MACOSX/DSE220x_PA7/._data3.txt',
 'DSE220x_PA7/data2.txt',
 '__MACOSX/DSE220x_PA7/._data2.txt',
 'DSE220x_PA7/data5.txt',
 '__MACOSX/DSE220x_PA7/._data5.txt',
 'DSE220x_PA7/data4.txt',
 '__MACOSX/DSE220x_PA7/._data4.txt',
 '__MACOSX/._DSE220x_PA7']

into /content/DSE220x_PA7 (or specify): 
/content/DSE220x_PA7/DSE220x_PA7 already exists
/content/DSE220x_PA7/DSE220x_PA7/kernel_SVM_and_Perceptron.ipynb already exists
/content/DSE220x_PA7/__MACOSX already exists
/content/DSE220x_PA7/__MACOSX/DSE220x_PA7 already exists
/content/DSE220x_PA7/__MACOSX/DSE220x_PA7/._kernel_SVM_and_Perceptron.ipynb already exists
/content/DSE220x_PA7/DSE220x_PA7/_nbgrader.log already exists
/content/DSE220x_PA7/__MACOSX/DSE220x_PA7/.__nbgrader.log already exists
/content/DSE220x_PA7/DSE220x_PA7/.ipynb_checkpoints already exists
/content/DSE220x_PA7/DSE220x_PA7/.ipynb_checkpoints/kernel_SVM_and_Perceptron-checkpoint.ipynb already exists
/content/DSE220x_PA7/__MACOSX/DSE220x_PA7/.ipynb_checkpoints already exists
/content/DSE220x_PA7/__MACOSX/DSE220x_PA7/.ipynb_checkpoints/._kernel_SVM_and_Perceptron-checkpoint.ipynb already exists
/content/DSE220x_PA7/__MACOSX/DSE220x_PA7/._.ipynb_checkpoints already exists
/content/DSE220x_PA7/DSE220x_PA7/data1.txt already 